In [1]:
from time import time
import pandas as pd
import numpy as np
import SARpy
import operator
from rdkit import Chem
from rdkit.Chem import Draw
from sklearn.model_selection import train_test_split
from pprint import pprint
from sklearn.metrics import confusion_matrix
import scipy.stats as stats

In [20]:
structures = pd.read_csv("./CAMDA_Model_Data/compounds_added_inactives.csv")
structures.rename(columns = {'standardized_smiles':'SMILES'}, inplace=True)
structures.head()

Unnamed: 0  PubChem_CID  Compound Name  \
0           0        34869     amineptine   
1           1         2717  chlormezanone   
2           2        65679       droxicam   
3           3        65869     ebrotidine   
4           4         4495     nimesulide   

                                            SMILES          MW  \
0               O=C(O)CCCCCCNC1c2ccccc2CCc2ccccc21  337.204179   
1                CN1C(=O)CCS(=O)(=O)C1c1ccc(Cl)cc1  273.022642   
2  CN1c2c(oc(=O)n(-c3ccccn3)c2=O)-c2ccccc2S1(=O)=O  357.041941   
3    NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1  475.975849   
4       CS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Oc1ccccc1  308.046692   

         vDILIConcern  
0  vMost-DILI-Concern  
1  vMost-DILI-Concern  
2  vMost-DILI-Concern  
3  vMost-DILI-Concern  
4  vMost-DILI-Concern

In [21]:
structures2 = pd.read_csv("./CAMDA_Model_Data/compounds_annotated_filtered.csv")
structures2.head()

Unnamed: 0   LTKBID PubChem_CID  Compound Name ApprovalYear  \
0           0  LT00045       34869     amineptine            .   
1           1  LT01201        2717  chlormezanone         1960   
2           2  LT01318       65679       droxicam            .   
3           3  LT01849       65869     ebrotidine            .   
4           4  LT00314        4495     nimesulide            .   

         DILIConcern  Severity Class Label Section        vDILIConcern  \
0  Most-DILI-Concern               8     Withdrawn  vMost-DILI-Concern   
1  Most-DILI-Concern               8     Withdrawn  vMost-DILI-Concern   
2  Most-DILI-Concern               8     Withdrawn  vMost-DILI-Concern   
3  Most-DILI-Concern               8     Withdrawn  vMost-DILI-Concern   
4  Most-DILI-Concern               8     Withdrawn  vMost-DILI-Concern   

  Greene_Annotation Sakatis_Annotation Xu_Annotation Zhu_Annotation  \
0                HH                  .      Positive        Postive   
1                 .                  .             .        Postive   
2                 .                  .             .              .   
3                 .                  .             .              .   
4                HH        Hepatotoxic      Positive        Postive   

                                              SMILES  \
0  C1CC2=CC=CC=C2C(C3=CC=CC=C31)[NH2+]CCCCCCC(=O)...   
1               CN1C(S(=O)(=O)CCC1=O)C2=CC=C(C=C2)Cl   
2  CN1C2=C(C3=CC=CC=C3S1(=O)=O)OC(=O)N(C2=O)C4=CC...   
3  C1=CC(=CC=C1S(=O)(=O)NC=NCCSCC2=CSC(=N2)N=C(N)...   
4  CS(=O)(=O)NC1=C(C=C(C=C1)[N+](=O)[O-])OC2=CC=C...   

  URL (accessed on 1/21/2016)  Version  \
0                         NaN        1   
1                         NaN        1   
2                         NaN        1   
3                         NaN        1   
4                         NaN        1   

                                          can_smis  \
0    O=C(O)CCCCCC[NH2+]C1c2ccccc2CCc2ccccc21.[Cl-]   
1                CN1C(=O)CCS(=O)(=O)C1c1ccc(Cl)cc1   
2  CN1c2c(oc(=O)n(-c3ccccn3)c2=O)-c2ccccc2S1(=O)=O   
3    NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1   
4       CS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Oc1ccccc1   

                               standardized_smiles  c_atoms       MW  
0               O=C(O)CCCCCCNC1c2ccccc2CCc2ccccc21       22  337.463  
1                CN1C(=O)CCS(=O)(=O)C1c1ccc(Cl)cc1       12  273.741  
2  CN1c2c(oc(=O)n(-c3ccccn3)c2=O)-c2ccccc2S1(=O)=O       16  357.347  
3    NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1       14  477.435  
4       CS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Oc1ccccc1       13  308.315

In [22]:
structures
structures['vDILIConcern'].replace(['vLess-DILI-Concern'], 'vMost-DILI-Concern', inplace = True)
structures['vDILIConcern'].replace(['sider_inactive'], 'vNo-DILI-Concern', inplace = True)
structures = structures[structures['vDILIConcern'] != 'Ambiguous DILI-concern']

In [23]:
structures
SMILES = structures.loc[:,['SMILES']]
#DILI = structures.loc[structures['vDILIConcern'] == 'vMost-DILI-Concern'].SMILES
#no_DILI = structures.loc[structures['vDILIConcern'] == 'vNo-DILI-Concern'].SMILES

In [24]:
structures

Unnamed: 0  PubChem_CID            Compound Name  \
0              0        34869               amineptine   
1              1         2717            chlormezanone   
2              2        65679                 droxicam   
3              3        65869               ebrotidine   
4              4         4495               nimesulide   
5              5         5591             troglitazone   
6              6        62959            trovafloxacin   
7              7         2081              alaproclate   
8              8      3086677  alatrofloxacin mesylate   
9              9        30951               alclofenac   
10            10        54897                  alpidem   
11            11         2313                 bendazac   
12            12        39941             benoxaprofen   
13            13       255968                benzarone   
14            14         2333            benzbromarone   
15            15         6237             benziodarone   
16            16        60726                bromfenac   
17            17         8593               cinchophen   
18            18        21382                clomacran   
19            19        33176               clometacin   
20            20         2898               cyclofenil   
21            21        40399                  exifone   
22            22        28858           fenclozic acid   
23            23         3351                 fipexide   
24            24         3474                glafenine   
25            25        15250                 ibufenac   
26            26         3748               iproniazid   
27            27        71169                isaxonine   
28            28       151166              lumiracoxib   
29            29         6179               mebanazine   
...          ...          ...                      ...   
1106        1106      9871419               ticagrelor   
1107        1107     11561674               apremilast   
1108        1108      9924495               perampanel   
1109        1109      9930048              vernakalant   
1110        1110      9966051                       Lu   
1111        1111     10077129                vorapaxar   
1112        1112     10096344              linagliptin   
1113        1113     15950376         18F-flutemetamol   
1114        1114     10178705             besifloxacin   
1115        1115     11658859               lorcaserin   
1116        1116     11001318                tafamidis   
1117        1117     11467166               aclidinium   
1118        1118     11304743                riociguat   
1119        1119     11501341              florbetaben   
1120        1120     11519069             umeclidinium   
1121        1121     23652731               eliglustat   
1122        1122     11949646            empagliflozin   
1123        1123     13559281               ulipristal   
1124        1124     15951529                  MDV3100   
1125        1125     24821094                PCI-32765   
1126        1126     19371515              alcaftadine   
1127        1127     24812758            canagliflozin   
1128        1128     24822371              florbetapir   
1129        1129     57396865              florbetapir   
1130        1130     24838347                 glucagon   
1131        1131     24846132               hetastarch   
1132        1132     24965990               suvorexant   
1133        1133     54675779          oxytetracycline   
1134        1134     53477714                  heparin   
1135        1135     53627505            hydroxypropyl   

                                                 SMILES          MW  \
0                    O=C(O)CCCCCCNC1c2ccccc2CCc2ccccc21  337.204179   
1                     CN1C(=O)CCS(=O)(=O)C1c1ccc(Cl)cc1  273.022642   
2       CN1c2c(oc(=O)n(-c3ccccn3)c2=O)-c2ccccc2S1(=O)=O  357.041941   
3         NC(N)=Nc1nc(CSCCN=CNS(=O)(=O)c2ccc(Br)cc2)cs1  475.975849   
4            CS(=O)(=O)Nc1ccc([N+](=O)[O-])cc1Oc1ccccc

In [25]:
def label (x):
    if x == 'vMost-DILI-Concern':
        return 1
    if x == 'vNo-DILI-Concern':
        return 0

In [26]:
SMILES['DILI_LABEL'] = structures['vDILIConcern'].apply(label)
SMILES.to_csv("Data_for_SARpy.csv")

In [27]:
data = pd.read_csv("Data_for_SARpy.csv")
x = data[['SMILES','DILI_LABEL']]
y = data['DILI_LABEL']
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.25, random_state=42, stratify=y) # statify means equal ratios classes in training and test sets
x_train.to_csv("Train_Data_for_SARpy.csv")
x_test.to_csv("Test_Data_for_SARpy.csv")

In [28]:
x_test.head()

SMILES  DILI_LABEL
267  NC(C(=O)N[C@@H]1C(=O)N2C(C(=O)O)=C(Cl)CS[C@H]1...           1
624                          CC(N)C12CC3CC(CC(C3)C1)C2           0
205            COC(=O)Nc1nc2ccc(C(=O)c3ccccc3)cc2[nH]1           1
522                                      O=C([O-])CCCO           0
99                      CNCC[C@@H](Oc1ccccc1C)c1ccccc1           1

In [29]:
filt1 = SARpy.Filter('DILI_LABEL',0, operator.eq)
filt2 = SARpy.Filter('DILI_LABEL',1, operator.eq)
dictionary = {'INACTIVE':filt1, 'ACTIVE':filt2}
dataset = SARpy.loadDataset("Train_Data_for_SARpy.csv",'csv',dictionary,'SMILES')
dataset_test = SARpy.loadDataset("Test_Data_for_SARpy.csv",'csv',dictionary,'SMILES')



Loading dataset...

 Read 690 molecular structures
('', 326, 'ACTIVE')
('', 364, 'INACTIVE')


Loading dataset...

 Read 230 molecular structures
('', 108, 'ACTIVE')
('', 122, 'INACTIVE')


In [48]:
SARpy.fragmentize(dataset,4,18)
rules = SARpy.extract(dataset, minHits = 5,minLR = 2, minPrecision = 0.1) # 'OPTIMAL'
SARpy.saveSmarts(rules,'ruleset.txt')



Fragmenting...	

 3218	substructures found...
 7250	substructures found...
 7974	substructures found...
 5472	substructures found...
 2710	substructures found...
 1004	substructures found...
 245	substructures found...
 42	substructures found...
 5	substructures found...
 0	substructures found...

FRAGMENTS: 27920

Evaluating fragments on the training set...

    -> elapsed time: 61.49 seconds
         fragmentation 56.70 seconds
              matching 4.80 seconds


Extracting rules...	

 16775 ACTIVE substructures
  1409 of which are potential alerts

 18053 INACTIVE substructures
  2192 of which are potential alerts

 Extracted:
 35	ACTIVE
 35	INACTIVE

RULES: 70

 -> time: 5.93 seconds


70 RULES have been saved


In [49]:
## Long winded way to get substructures for DILI (not for noDILI)

accuracy_list = []
sensitivity_list = []
specificity_list = []
SMARTS_list = []
LR_list = []
Target_list = []
test_comp_matched_list = []
for rule in rules:
    myrules = [rule,rule]
    SARpy.saveSmarts(myrules,'ruleset.txt')
    alerts = pd.read_csv("ruleset.txt", sep='\t')
    alerts = alerts.ix[1]
    #print(alerts)
    SMARTS = alerts['SMARTS']
    TARGET = alerts['Target']
    Target_list.append(TARGET)
    if TARGET == 'ACTIVE': # only DILI active substructures
        
    
        LR = alerts['Training LR']
        pred = SARpy.predict(myrules,dataset_test)
        if pred >= 1:
            accuracy,sensitivity,specificity = SARpy.validate(dataset_test)
                #print(accuracy,sensitivity,specificity,LR, SMARTS)
            pred = (pred/167.0)*100
            test_comp_matched_list.append((pred))
            accuracy_list.append(accuracy)
            sensitivity_list.append(sensitivity)
            specificity_list.append(specificity)
            SMARTS_list.append(SMARTS)
            LR_list.append(LR)
            
            
        
    else:
        continue
        
        
df = pd.DataFrame()
df['SMARTS'] = SMARTS_list
df['TYPE'] = Target_list
df['Accuracy'] = accuracy_list
df['Sensitivity'] = sensitivity_list
df['Specificity'] = specificity_list
df['LR_training'] = LR_list
df['Percent_matched_test_compounds'] = test_comp_matched_list
df

IndentationError: expected an indented block (<ipython-input-49-c295c7abb14a>, line 22)

In [ ]:
df.head()

In [ ]:
## Tabulate results


#https://www.rdkit.org/docs/GettingStartedInPython.html#substructure-searching

df2 = pd.DataFrame(index = x['SMILES'])
df2['DILI_LABEL'] = x['DILI_LABEL'].to_list()
train_list = []
#test_list = []
for index, row in x.iterrows():
    a = row['SMILES']
    if a in x_train['SMILES'].tolist():
        train_list.append('yes')
        #test_list.append('no')
    if a in x_test['SMILES'].tolist():
        train_list.append('no')
        #test_list.append('yes')

print(len(train_list))
        
df2['Train_Set'] = train_list
#df2['Test_Set'] = test_list

h = 0 # count TPs
l = 0 # count FPs
PPV_dict = {}
for index, row in df.iterrows():
    i=0
    j=0
    k=0
    hit_list = []
    smart = row['SMARTS']
    for index, row in x.iterrows(): # Test set
        label = row['DILI_LABEL']
        a = row['SMILES']   
        m = Chem.MolFromSmiles(a)
        patt = Chem.MolFromSmarts(smart)
        if m.HasSubstructMatch(patt) is True:
            i+=1
        if m.HasSubstructMatch(patt) is True and label == 1: # True positive
            #print('TP')
            j+=1
        if m.HasSubstructMatch(patt) is True and label == 0: # False positive
            #print('FP')
            k+=1
        if m.HasSubstructMatch(patt) is True:
            hit_list.append(1)
        if m.HasSubstructMatch(patt) is False:
            hit_list.append('0')
    
    PPV = (j/(j+k*1.0))
    print('{0:.3f}'.format(PPV)) # positive predictive value (most important for substructures to classify DILI)
    
    if PPV >= 0.8: # filter for PPV
    
        df2[smart] = hit_list
        print(i,j,k,len(x_test['SMILES']),len(x_test.loc[x_test['DILI_LABEL']==1]),len(x_test.loc[x_test['DILI_LABEL']==0]))
        h+=j
        l+=k
        PPV_dict[smart] = PPV
    else:
        continue
    
print(h,l) # number of TPs and FPs

In [ ]:
PPV_dict = dict((k, v) for k, v in PPV_dict.items() if v >= 0.8)
print(PPV_dict)
df2

In [44]:
for index, row in df.iterrows():
    

IndentationError: expected an indented block (<ipython-input-44-ad68754b63a9>, line 2)

In [36]:
df2.to_csv("Substructure_Matrix.csv")